In [ ]:
from src.datasets.patch_tumor_classify_datamodule import BraTSDataModule
import yaml

import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
%matplotlib inline


In [ ]:
with open('/home/sanyal/Projects/BraTS3DDiff/configs/data/brats23_patch_tumor_classify.yaml') as f:
    cfg = yaml.safe_load(f)
cfg.pop('_target_')
cfg['data_dir'] = '/home/sanyal/Projects/BraTS3DDiff/data/BraTS-Data/BraTS2023-GLI'
cfg['patch_sizes'] = [2,4,8,16,32]
cfg['batch_size'] = 1
a = BraTSDataModule(**cfg)

train_data = a.train_dataloader()
data = iter(train_data).__next__()
image = data["image"]
print("train loader data", image.shape)
mask = data["mask"]
print("train loader mask", mask.shape)
b,c,w,h,d = mask.shape
patch_labels = data['patch_tumor_labels']
print("train loader patch vols", [patch_labels[i].shape for i in cfg['patch_sizes']])
fig, axes = plt.subplots(nrows=1, ncols=7, figsize=(15, 5))
axes[0].imshow(image[0,0,:,:,78],cmap='gray')
axes[1].imshow(mask[0,0,:,:,78],cmap='gray')
for i,(_,patch_label) in enumerate(patch_labels.items()):
    axes[i+2].imshow(patch_label[0,0,:,:,patch_label.shape[-1]//2],cmap='gray')

In [ ]:
patch_label_32 = patch_labels[32]
patch_label_32.shape

In [ ]:
mask.shape

In [ ]:
pos_indices = patch_label_32.nonzero(as_tuple=True)
pos_indices

In [ ]:
patch_label_32.nonzero(as_tuple=False)


In [ ]:
pos_indices[0].shape

In [ ]:
mask_patches = mask.view(1,3,4,32,4,32,4,32)
mask_patches_pos = mask_patches[pos_indices[0], :, pos_indices[2], :,  pos_indices[3], :, pos_indices[4], :]
mask_patches_pos.shape

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
axes.imshow(mask[0,0,:,:,42],cmap='gray')

In [ ]:
mask.shape

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
axes.imshow(mask_patches_pos[8,0,:,:,10],cmap='gray')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
axes.imshow(mask_patches_pos[2,0,:,:,31],cmap='gray')

In [ ]:
mask_patches_pos.shape

In [ ]:
fill_indices = torch.nonzero(patch_label_32, as_tuple=False)

fill_indices

In [ ]:
predicted_mask = torch.zeros((1, 3, 128, 128, 128))
P=32

# Iterate over fill_indices and scatter the predicted masks into the full tensor
for i in range(fill_indices.size(0)):
    b, _, w_idx, h_idx, d_idx = fill_indices[i]
    predicted_mask[b, :, w_idx * P:(w_idx + 1) * P,
                         h_idx * P:(h_idx + 1) * P,
                         d_idx * P:(d_idx + 1) * P] = mask_patches_pos[i].unsqueeze(0)

In [ ]:
predicted_mask.shape

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
axes.imshow(mask[0,0,:,:,78],cmap='gray')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
axes.imshow(predicted_mask[0,0,:,:,78],cmap='gray')

In [ ]:
torch.equal(predicted_mask, mask)


In [ ]:
import numpy as np
np.prod(list(predicted_mask.shape))

In [ ]:
mask_patches.shape

In [ ]:
list(enumerate(patch_labels.items()))predicted_mask[indices[0] :, indices[2] * P: (indices[2] + 1) * P, indices[3] * P: (indices[3] + 1) * P, indices[4] * P: (indices[4] + 1) * P] = patch_embeddings_extracted

In [ ]:
patch_vol

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=7, figsize=(15, 5))
axes[0].imshow(image[0,0,:,:,78],cmap='gray')
axes[1].imshow(mask[0,0,:,:,78],cmap='gray')
for i,patch_vol in enumerate(patch_vols):
    axes[i+2].imshow(patch_vol[0,0,:,:,2],cmap='gray')


fig.tight_layout()

In [ ]:
#visualize patches
im_channel=0
depth=78
image = image[0,im_channel,:,:,depth]
mask = mask[0,:,:,:,depth]
print(image.shape,mask.shape)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 3))
axes[0].imshow(image,cmap='gray')
axes[1].imshow(mask[0],cmap='gray')
fig.tight_layout()

In [ ]:
with open('/home/sanyal/Projects/BraTS3DDiff/configs/data/brats23_patch_tumor_classify.yaml') as f:
    cfg = yaml.safe_load(f)
cfg.pop('_target_')
cfg['data_dir'] = '/home/sanyal/Projects/BraTS3DDiff/data/BraTS-Data/BraTS2023-GLI'
cfg['patch_sizes'] = [2,4,8,16,32]
cfg['batch_size'] = 1
a = BraTSDataModule(**cfg)

train_data = a.train_dataloader()
data = iter(train_data).__next__()
image = data["image"]
print("train loader data", image.shape)
mask = data["mask"]
print("train loader mask", mask.shape)
b,c,w,h,d = mask.shape
patch_labels = data['patch_tumor_labels']
print("train loader patch vols", [patch_labels[i].shape for i in cfg['patch_sizes']])

#visualize patches
im_channel=0
depth=78
image = image[0,im_channel,:,:,depth]
mask = mask[0,:,:,:,depth]
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 5))
axes[0].imshow(image,cmap='gray')
for i in range(1,4):
    axes[i].imshow(mask[i-1,:,:])
print(image.shape,mask.shape)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 3))
axes[0].imshow(image,cmap='gray')
axes[1].imshow(mask[0],cmap='gray')
fig.tight_layout()

w,h = image.shape
c,w,h = mask.shape
patch_size = cfg['patch_sizes'][-2]

img_patches = image.view(w//patch_size,patch_size,h//patch_size,patch_size)
mask_patches = mask.view(c,w//patch_size,patch_size,h//patch_size,patch_size)
mask_patches_vol = mask_patches[0].sum(dim=(1,3))
mask_patches_vol[mask_patches_vol > 0] = 1
mask_patches_vol[mask_patches_vol == 0] = 0


fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                nrows_ncols=(w//patch_size, h//patch_size),  # creates 2x2 grid of Axes
                axes_pad=0.1,  # pad between Axes in inch.
                )

grid_w,grid_h = (w//patch_size, h//patch_size)
#indices = torch.cartesian_prod(torch.arange(grid_w),torch.arange(grid_h))
for idx, ax in enumerate(grid):
    # Iterating over the grid returns the Axes.
    patch_index = (idx//grid_h, idx%grid_h)
    #print('patch_index_1',indices[idx])
    ax.imshow(img_patches[patch_index[0],:,patch_index[1],:],cmap='gray')

plt.show()
fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                nrows_ncols=(w//patch_size, h//patch_size),  # creates 2x2 grid of Axes
                axes_pad=0.1,  # pad between Axes in inch.
                )

grid_w,grid_h = (w//patch_size, h//patch_size)
#indices = torch.cartesian_prod(torch.arange(grid_w),torch.arange(grid_h))
for idx, ax in enumerate(grid):
    # Iterating over the grid returns the Axes.
    patch_index = (idx//grid_h, idx%grid_h)
    #print('patch_index_1',indices[idx])
    ax.imshow(mask_patches[0, patch_index[0],:,patch_index[1],:],cmap='gray')
plt.show()

fig,ax = plt.subplots(1,1)
ax.imshow(mask_patches_vol,cmap='gray')

In [ ]:
with open('/home/sanyal/Projects/BraTS3DDiff/configs/data/brats23_patch_tumor_classify.yaml') as f:
    cfg = yaml.safe_load(f)
cfg.pop('_target_')
cfg['data_dir'] = '/home/sanyal/Projects/BraTS3DDiff/data/BraTS-Data/BraTS2023-GLI'
cfg['patch_sizes'] = [2,4,8,16,32]
cfg['batch_size'] = 1
a = BraTSDataModule(**cfg)

train_data = a.train_dataloader()
data = iter(train_data).__next__()
image = data["image"]
print("train loader data", image.shape)
mask = data["mask"]
print("train loader mask", mask.shape)
b,c,w,h,d = mask.shape
patch_labels = data['patch_tumor_labels']
print("train loader patch vols", [patch_labels[i].shape for i in cfg['patch_sizes']])

#visualize patches
im_channel=0
depth=78
image = image[0,im_channel,:,:,depth]
mask = mask[0,:,:,:,depth]
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 5))
axes[0].imshow(image,cmap='gray')
for i in range(1,4):
    axes[i].imshow(mask[i-1,:,:],cmap='gray')
print(image.shape,mask.shape)



In [ ]:
mask.shape


In [ ]:
mask_0 = mask[0,:,:]
w,h = mask_0.shape

fig, axes = plt.subplots(nrows=1, ncols=7, figsize=(25, 10))
axes[0].imshow(image,cmap='gray')
axes[1].imshow(mask_0,cmap='gray')
for i in range(len(cfg['patch_sizes'])):
    patch_size = cfg['patch_sizes'][i]
    mask_patches = mask_0.view(w//patch_size,patch_size,h//patch_size,patch_size)
    mask_patches_label = mask_patches.sum(dim=(1,3))
    mask_patches_label[mask_patches_label > 0] = 1
    mask_patches_label[mask_patches_label == 0] = 0

    axes[i+2].imshow(mask_patches_label,cmap='gray')


fig.tight_layout()

In [ ]:
mask_patches_label.shape

In [ ]:
plt.imshow(mask_patches_label)

In [ ]:
mask_patches_label

In [ ]:
#filter indices
true_indices = mask_patches_label.nonzero(as_tuple=True)
true_indices

In [ ]:
plt.imshow(mask_0)

In [ ]:
mask_0.shape

In [ ]:
mask_0_patches = mask_0.view(4,32,4,32).permute(1,3,0,2).contiguous().view(32*32,4,4)
mask_0_patches.shape

In [ ]:
mask_0_patches_ = mask_0_patches.permute(1,2,0).contiguous()
mask_0_patches_.shape

In [ ]:
fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                nrows_ncols=(4, 4),  # creates 2x2 grid of Axes
                axes_pad=0.1,  # pad between Axes in inch.
                )

grid_w,grid_h = (4, 4)
#downsample into 4x4 grid
mask_0_patches = mask_0.view(4,32,4,32)
#flatten grid
mask_0_patches_ = mask_0_patches.permute(0,2,1,3).contiguous().view(16,32,32)

#indices = torch.cartesian_prod(torch.arange(grid_w),torch.arange(grid_h))
for idx, ax in enumerate(grid):
    # Iterating over the grid returns the Axes.
    ax.imshow(mask_0_patches_[idx],cmap='gray')
plt.show()

In [ ]:
fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                nrows_ncols=(4, 4),  # creates 2x2 grid of Axes
                axes_pad=0.1,  # pad between Axes in inch.
                )

grid_w,grid_h = (4, 4)
#downsample into 4x4 grid
mask_0_patches = mask_0.view(4,32,4,32)

#indices = torch.cartesian_prod(torch.arange(grid_w),torch.arange(grid_h))
for idx, ax in enumerate(grid):
    # Iterating over the grid returns the Axes.
    patch_index = (idx//grid_h, idx%grid_h)
    #print('patch_index_1',indices[idx])
    ax.imshow(mask_0_patches[patch_index[0],:,patch_index[1],:],cmap='gray')
plt.show()

In [ ]:
mask_patches_label

In [ ]:
fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                nrows_ncols=(4, 4),  # creates 2x2 grid of Axes
                axes_pad=0.1,  # pad between Axes in inch.
                )

grid_w,grid_h = (4, 4)
mask_0_patches = mask_0.view(4,32,4,32)
#indices = torch.cartesian_prod(torch.arange(grid_w),torch.arange(grid_h))
for idx, ax in enumerate(grid):
    # Iterating over the grid returns the Axes.
    patch_index = (idx//grid_h, idx%grid_h)
    #print('patch_index_1',indices[idx])
    ax.imshow(mask_0_patches[patch_index[0],:,patch_index[1],:],cmap='gray')
plt.show()

In [ ]:
mask_patches_label.shape

In [ ]:
mask_0_patches.shape

In [ ]:
mask_0_ = mask_0_patches.view(128,128)
plt.imshow(mask_0_)

In [ ]:
plt.imshow(mask_0_*x)

In [ ]:
x=mask_patches_label.view(4,1,4,1).expand(4,32,4,32).contiguous().view(128,128)
plt.imshow(x)

In [ ]:
#reshape mask_0
mask_0_patches = mask_0.view(4,32,4,32)
mask_0_patches.shape

In [ ]:
true_indices

In [ ]:
#subset mask_0_patches using true_indices
mask_0_patches_true = mask_0_patches[true_indices[0],:,true_indices[1],:]
mask_0_patches_true.shape

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(25, 10))
for i in range(mask_0_patches_true.shape[0]):
    axes[i].imshow(mask_0_patches_true[i],cmap='gray')

In [ ]:
true_indices

In [ ]:
mask_0.shape

In [ ]:
pred_mask_0 = torch.zeros((128,128))
pred_mask_0.shape

In [ ]:
fill_indices = mask_patches_label.nonzero(as_tuple=False)
fill_indices

In [ ]:
for i in range(fill_indices.shape[0]):
    fill_idx = fill_indices[i]
    pred_mask_0[fill_idx[0]*32:(fill_idx[0]+1)*32,fill_idx[1]*32:(fill_idx[1]+1)*32] = mask_0_patches_true[i]

In [ ]:
plt.imshow(pred_mask_0,cmap='gray')

In [ ]:
torch.equal(pred_mask_0,mask_0)

In [ ]:
mask_patches_label

In [ ]:
w,h = image.shape
c,w,h = mask.shape
patch_size = cfg['patch_sizes'][-1]

img_patches = image.view(w//patch_size,patch_size,h//patch_size,patch_size)
mask_patches = mask.view(c,w//patch_size,patch_size,h//patch_size,patch_size)
mask_patches_vol = mask_patches[0].sum(dim=(1,3))/(patch_size*patch_size)


fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                nrows_ncols=(w//patch_size, h//patch_size),  # creates 2x2 grid of Axes
                axes_pad=0.1,  # pad between Axes in inch.
                )

grid_w,grid_h = (w//patch_size, h//patch_size)
#indices = torch.cartesian_prod(torch.arange(grid_w),torch.arange(grid_h))
for idx, ax in enumerate(grid):
    # Iterating over the grid returns the Axes.
    patch_index = (idx//grid_h, idx%grid_h)
    #print('patch_index_1',indices[idx])
    ax.imshow(img_patches[patch_index[0],:,patch_index[1],:],cmap='gray')

plt.show()
fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                nrows_ncols=(w//patch_size, h//patch_size),  # creates 2x2 grid of Axes
                axes_pad=0.1,  # pad between Axes in inch.
                )

grid_w,grid_h = (w//patch_size, h//patch_size)
#indices = torch.cartesian_prod(torch.arange(grid_w),torch.arange(grid_h))
for idx, ax in enumerate(grid):
    # Iterating over the grid returns the Axes.
    patch_index = (idx//grid_h, idx%grid_h)
    #print('patch_index_1',indices[idx])
    ax.imshow(mask_patches[0, patch_index[0],:,patch_index[1],:],cmap='gray')
plt.show()

fig,ax = plt.subplots(1,1)
ax.imshow(mask_patches_vol,cmap='gray')

In [ ]:
mask_patches_vol

In [ ]:


fig, axes = plt.subplots(nrows=1, ncols=7, figsize=(15, 5))
axes[0].imshow(image,cmap='gray')
axes[1].imshow(mask[1],cmap='gray')
for i in range(len(cfg['patch_sizes'])):
    patch_size = cfg['patch_sizes'][i]
    mask_patches = mask.view(c,w//patch_size,patch_size,h//patch_size,patch_size)
    mask_patches_vol = mask_patches[1].sum(dim=(1,3))
    axes[i+2].imshow(mask_patches_vol,cmap='gray')


fig.tight_layout()


In [ ]:
mask_patches_vol

In [ ]:
mask.shape

In [ ]:
patch_size=32
mask_patches = mask.view(c,w//patch_size,patch_size,h//patch_size,patch_size)
mask_patches_vol = mask_patches[0].sum(dim=(1,3))

In [ ]:
mask_patches_vol

In [ ]:
32*32

In [ ]:
mask_patches_vol